Criteria:
1. Age between 18 and 90 
2. Exclude patients with pre-existing end stage renal disease
3. Exclude patients who have undergone any dialysis procedure or renal transplantation (RRT) prior to the visit  
4. Exclude patients who eGFR < 15 mL/min/1.73 m^2 or baseline SCr > 3.5 mg/dL
5. Hospitalized for at least 2 days 
6. Exclude non-AKI patients
7. Exclude AKI progression patients
8. SCr trajectories satisfy the requirements
9. Each patient unqiue (i.e. one patient one encounter)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Create a Sreening Tracking DataFrame

In [ ]:
%store -r ct_names
%store -r raw_path
%store -r pat_id_cols

#pandas can show all the columns
pd.set_option("...", None)

In [ ]:
records_num_df = pd.DataFrame(0, index = ["...", 
                                          "...",
                                          "...",
                                          "...",
                                          "...",
                                          "...",
                                          "...",
                                          "...",
                                          "..."], 
                              columns = ct_names)

In [ ]:
def fill_in_record_num_df(screen_item, ct_names, onset_df):
    # fill in table
    screen_item = screen_item
    for ct_name in ct_names:
        ct_enc_n = len(onset_df[onset_df.CENTER_NAME == ct_name])
        print("..."%(ct_name, screen_item, ct_enc_n))
        records_num_df.loc[screen_item, ct_name] = ct_enc_n

# Read AKI Onset Table

In [ ]:
onset_df = pd.read_csv("...")

In [ ]:
# format type
onset_df[pat_id_cols] = onset_df[pat_id_cols].astype(str)
date_cols = ["...", "...", "...", "...", "..."]
for col in date_cols:
    onset_df[col] = pd.to_datetime(onset_df[col], format = "...")

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

# Read Demographics and Screen Patients

In [ ]:
import nbimporter
from A_Label_AKI_Onsets import read_and_format_DEMO
%store -r race_mapping

In [ ]:
DEMO_df = read_and_format_DEMO(ct_names, raw_path, race_mapping)

In [ ]:
# format type
DEMO_df[pat_id_cols + ["...", "..."]] = DEMO_df[pat_id_cols + ["...", "..."]].astype(str)

In [ ]:
onset_df = onset_df.merge(DEMO_df, on = pat_id_cols, how = "...")
onset_df = onset_df.drop_duplicates(pat_id_cols)

In [ ]:
# sanity check
assert(onset_df["..."].isna().mean() == 0)
assert(onset_df["..."].isna().mean() == 0)
assert(onset_df["..."].isna().mean() == 0)

In [ ]:
onset_df

In [ ]:
# criteria 1: age between 18 and 90
onset_df = onset_df[(onset_df.AGE >= 18) & (onset_df.AGE < 90)]

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

# Read Diagnosis and Screen Patients

Remove dialysis, RRT and ESRD encounters from the dianosis side  

In [ ]:
from A_Label_AKI_Onsets import read_DX

In [ ]:
DX_dict = read_DX(ct_names, raw_path)

In [ ]:
ESRD_dia_RRT_codes = {
    "...": ["...",
             "...","...","...", "...", 
             "...","...","...","...","...","...", "..."],
    
    "...": ["...",
              "...","...","...","...","...", "...",
              "...","...","...","...","...","...","...","...","...","...","...", "..."],
    
    "...": [str(cpt) for cpt in range(90935, 91000)] + \
    ["...","...","...","...","...","...","...","...","...","...","...","..."]
}

In [ ]:
# return encounters related to the input code dict
def get_enc_by_DX_code(DX_dict, pat_df, code_dict, code_types, pat_id_cols):
    processed_DX_dict = dict()
    ct_missing_DX_DATE = ["...", "...", "..."]
    
    for ct_name, DX_df in tqdm(DX_dict.items()):
        # format type
        DX_df[["...", "...", "..."]] = DX_df[["...", "...", "..."]].astype(str)
        DX_df["..."] = DX_df["..."].replace("...", "...")
        DX_df["..."] = DX_df["..."].replace("...", "...")
        DX_df["..."] = DX_df["..."].replace("...", "...")
        
        # we only care about code-related DX
        DX_in_codes = []
        for code_type in code_types:
            DX_df_temp = DX_df[(DX_df.DX_TYPE == code_type) & (DX_df.DX.isin(code_dict[code_type]))]
            DX_in_codes.append(DX_df_temp)
            
        DX_df = pd.concat(DX_in_codes, axis = 0)
        
        pat_ct_df = pat_df[pat_df.CENTER_NAME == ct_name]
        pat_ct_df = pat_ct_df.merge(DX_df[["...", "...", "...", "...", "..."]], 
                                    on = "...", how = "...")
        
        #drop rows do not involed in the codes
        pat_ct_df.dropna(subset=["..."], inplace = True)
        
        
        # format time cols so that we can filter "..." dx later
        if ct_name not in ct_missing_DX_DATE:
            pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
            pat_ct_df["..."] = pat_ct_df["..."].dt.strftime("...")
            pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
        else:
            pat_ct_df.loc[:, "..."] = pat_ct_df.loc[:, "..."] + \
            pd.to_timedelta(pat_ct_df.loc[:, "..."], unit="...")

        # require that it is "...", filter "..." dx
        pat_ct_df = pat_ct_df[pat_ct_df.DX_DATE < pat_ct_df.ADMIT_DATE]
        
        # keep useful info
        pat_ct_df = pat_ct_df[pat_id_cols]
        processed_DX_dict[ct_name] = pat_ct_df
        
    return processed_DX_dict

In [ ]:
enc_to_remove_DX_dict = get_enc_by_DX_code(DX_dict, onset_df, ESRD_dia_RRT_codes, ["...", "..."], pat_id_cols)

In [ ]:
# concat them together 
enc_to_remove_all = []
for df in enc_to_remove_DX_dict.values():
    enc_to_remove_all.append(df)

enc_to_remove_all = pd.concat(enc_to_remove_all, axis = 0)

In [ ]:
# remove pat_id_cols matched rows from onset_df
merged_df = onset_df.merge(enc_to_remove_all, on=pat_id_cols, how="...", indicator=True)
onset_df = merged_df[merged_df["..."] == "..."].drop(columns="...")

# Read Procedures and Screen Patients

In [ ]:
from A_Label_AKI_Onsets import get_data_path

In [ ]:
#read patients' Procedure data
def read_procedures(ct_names, raw_path):
    PX_dict = dict()
    use_cols = ["...", "...", "...", "..."]
    
    for ct_name in tqdm(ct_names):
        
        data_path = get_data_path(ct_name, raw_path)
        
        if (ct_name == "...") or (ct_name == "...") or (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
            
        elif (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
            
        elif (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", delimiter = "...")
            PX_df.columns = [col.upper() for col in PX_df.columns] 
            PX_df = PX_df[use_cols]
            
        elif (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
            
        elif (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
            
        elif (ct_name == "..."):
            PX_df = pd.read_csv(data_path + "...", 
                                 delimiter = "...", 
                                 usecols = ["...", "..."+PD.DATE_SHIFT"...", "...","..."])
            PX_df.columns = use_cols

        PX_dict[ct_name] = PX_df
        
    return PX_dict

In [ ]:
PX_dict = read_procedures(ct_names, raw_path)

In [ ]:
# return encounters related to the input code dict
def get_enc_by_PX_code(PX_dict, pat_df, code_dict, code_types, pat_id_cols):
    processed_PX_dict = dict()
    
    for ct_name, PX_df in tqdm(PX_dict.items()):
        # format type
        PX_df[["...", "...", "..."]] = PX_df[["...", "...", "..."]].astype(str)
        PX_df["..."] = PX_df["..."].replace("...", "...")
        PX_df["..."] = PX_df["..."].replace("...", "...")
        PX_df["..."] = PX_df["..."].replace("...", "...")
        
        # we only care about code-related PX, after that we format time
        PX_in_codes = []
        for code_type in code_types:
            PX_df_temp = PX_df[(PX_df.PX_TYPE == code_type) & (PX_df.PX.isin(code_dict[code_type]))]
            PX_in_codes.append(PX_df_temp)
            
        PX_df = pd.concat(PX_in_codes, axis = 0)
        
        pat_ct_df = pat_df[pat_df.CENTER_NAME == ct_name]
        pat_ct_df = pat_ct_df.merge(PX_df, on = "...", how = "...")
        
        #drop rows do not involed in the codes
        pat_ct_df.dropna(subset=["..."], inplace = True)
        
        
        # format time cols so that we can filter "..." dx later
        pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
        pat_ct_df["..."] = pat_ct_df["..."].dt.strftime("...")
        pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")


        # require that it is "...", filter "..." px
        pat_ct_df = pat_ct_df[pat_ct_df.PX_DATE < pat_ct_df.ADMIT_DATE]
        
        # keep useful info
        pat_ct_df = pat_ct_df[pat_id_cols]
        processed_PX_dict[ct_name] = pat_ct_df
        
    return processed_PX_dict

In [ ]:
enc_to_remove_PX_dict = get_enc_by_PX_code(PX_dict, onset_df, 
                                           ESRD_dia_RRT_codes, 
                                           ["...", "...", "..."],  
                                           pat_id_cols)

In [ ]:
# concat them together 
enc_to_remove_all = []
for df in enc_to_remove_PX_dict.values():
    enc_to_remove_all.append(df)

enc_to_remove_all = pd.concat(enc_to_remove_all, axis = 0)

In [ ]:
# remove pat_id_cols matched rows from onset_df
merged_df = onset_df.merge(enc_to_remove_all, on=pat_id_cols, how="...", indicator=True)
onset_df = merged_df[merged_df["..."] == "..."].drop(columns="...")

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

In [ ]:
onset_df

# Screen Patients by SCr Baseline and eGFR

In [ ]:
#calculate eGFR, based on SCr baseline
def calculate_ckd_epi(row):
    
    age = row["..."]
    gender = row["..."]
    race = row["..."]
    SCr = row["..."]
    
    # Constants for the CKD-EPI formula
    k = 0.7 if gender == "..." else 0.9
    alpha = -0.329 if gender == "..." else -0.411
    
    # Calculate the eGFR
    min_term = min(SCr / k, 1) ** alpha
    max_term = max(SCr / k, 1) ** -1.209
    age_term = 0.993 ** age
    
    # Gender and ethnicity adjustments
    gender_term = 1.018 if gender == "..." else 1
    african_american_term = 1.159 if race == "..." else 1
    
    eGFR = 141 * min_term * max_term * age_term * gender_term * african_american_term
    
    return eGFR

In [ ]:
onset_df["..."] = onset_df.apply(calculate_ckd_epi, axis = 1)

In [ ]:
# Patients with SCr baseline > 3.5 mg/dL or eGFR < 15 mL/min/1.73 m^2 excluded 
onset_df = onset_df[(onset_df.EGFR > 15) & (onset_df.BASELINE_SCR < 3.5)]

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

# Screen Patients by Hospitalization and AKI Progression

In [ ]:
onset_df["..."] = (onset_df["..."] - onset_df["..."]).dt.days

In [ ]:
# hospitalized for at least 2 days
onset_df = onset_df[onset_df.HOSPITALIZATION_LEN >= 2]

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

In [ ]:
# filter out non-AKI patients 
onset_df = onset_df[onset_df.NONAKI_SINCE_ADMIT == False]

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

In [ ]:
# AKI progressed patients excluded
onset_df = onset_df[(onset_df.AKI2_ONSET.isna()) & (onset_df.AKI3_ONSET.isna())]

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df)

# Sanity Check before Merging SCr

In [ ]:
#no dup encounter
df_for_check_dup = onset_df.drop_duplicates(pat_id_cols)
assert(len(df_for_check_dup) == len(onset_df))

#AKI1_ONSET between ADMIT_DATE and DISCHARGE_DATE

assert((onset_df["..."] >= onset_df["..."]).all())
assert((onset_df["..."] <= onset_df["..."]).all())

#each center row number above zero
for ct_name in ct_names:
    assert(len(onset_df[onset_df.CENTER_NAME == ct_name]) > 0)

#no nan in DEMO
assert(onset_df["..."].isna().mean() == 0)
assert(onset_df["..."].isna().mean() == 0)
assert(onset_df["..."].isna().mean() == 0)

# Read SCr Trajectories

In [ ]:
from A_Label_AKI_Onsets import read_and_format_SCR

In [ ]:
SCR_df = read_and_format_SCR(ct_names, raw_path)

In [ ]:
# multiple measurements on the same day averaged
SCR_df = SCR_df.sort_values(by=["...", "...", "..."])
SCR_df = SCR_df.groupby(["...", "...", "..."]).RESULT_NUM.mean().reset_index()

In [ ]:
onset_df_id = onset_df[pat_id_cols + ["..."]].copy(deep = True)
onset_SCR = onset_df_id.merge(SCR_df, on = ["...", "..."], how = "...")

In [ ]:
# get 7 days window before onset and 7 days window after onset
onset_SCR = onset_SCR[(onset_SCR.SPECIMEN_DATE >= (onset_SCR.AKI1_ONSET - pd.Timedelta(days=6))) & \
                     (onset_SCR.SPECIMEN_DATE <= (onset_SCR.AKI1_ONSET + pd.Timedelta(days=7)))]

In [ ]:
onset_SCR["..."] = (onset_SCR["..."] - onset_SCR["..."]).dt.days

In [ ]:
onset_SCR_unique_ids = onset_SCR[pat_id_cols]
onset_SCR_unique_ids = onset_SCR_unique_ids.drop_duplicates().reset_index(drop = True)

In [ ]:
SCR_window = np.arange(-6, 8) 
multi_index = pd.MultiIndex.from_product([onset_SCR_unique_ids.index, SCR_window], 
                                         names=["...", "..."]).to_frame(index=False)
full_SCR_traj = multi_index.join(onset_SCR_unique_ids, on="...")
full_SCR_traj = full_SCR_traj.drop(columns=["..."])

In [ ]:
full_SCR_traj = full_SCR_traj.merge(onset_SCR[pat_id_cols + ["...", "..."]],
                                   on = pat_id_cols + ["..."],
                                   how = "...")

In [ ]:
full_SCR_traj

In [ ]:
# use pat_id_cols as index, DAYS_TO_ONSET as cols and RESULT_NUM as values
full_SCR_traj = full_SCR_traj.pivot_table(index = pat_id_cols, 
                                     columns="...", 
                                     values="...").reset_index()

# reset index name
full_SCR_traj.columns.name = None

# Screen Patients by SCr Trajectories Requirements

1. Days -2, -1, 0 must not be nan, otherwise excluded.  
2. At most 2 nan in total, otherwise excluded.

In [ ]:
window_before_and_onset = [i for i in range(-6, 1)]
window_after_onset = [i for i in range(1, 8)]

In [ ]:
# Condition 1
condition1 = full_SCR_traj[[-2, -1, 0]].notna().all(axis=1)

# Condition 2
condition2 = full_SCR_traj[[-6, -5, -4, -3]].isna().sum(axis=1) <= 2

full_SCR_traj_filtered = full_SCR_traj[condition1 & condition2]

In [ ]:
# linear interpolation
full_SCR_traj_filtered.loc[:, window_before_and_onset] = \
full_SCR_traj_filtered[window_before_and_onset].interpolate(method="...", axis=1)

# backward fill
full_SCR_traj_filtered.loc[:, window_before_and_onset] = full_SCR_traj_filtered[window_before_and_onset].fillna(method="...", axis = 1)

assert(full_SCR_traj_filtered[window_before_and_onset].notna().all(axis = 1).all())

In [ ]:
# merge other info back 
onset_df_final = full_SCR_traj_filtered.merge(onset_df, on = pat_id_cols, how = "...")

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df_final)

# Take the Earliest Encounter of Each Patient and Final Check

In [ ]:
onset_df_final = onset_df_final.sort_values(by=["...", "...", "..."], ascending=True)

In [ ]:
onset_df_final_unq = onset_df_final.groupby(["...", "..."]).first().reset_index()

In [ ]:
fill_in_record_num_df("...", ct_names, onset_df_final_unq)

In [ ]:
onset_df_final_unq.to_csv("...", index=False)

In [ ]:
# save for future use 
window_before_and_onset = [str(i) for i in window_before_and_onset]
window_after_onset = [str(i) for i in window_after_onset]
window_full = window_before_and_onset + window_after_onset

%store window_before_and_onset
%store window_after_onset
%store window_full

# Show Screening Tracking Table

In [ ]:
def add_percentage(col):
    total = col.iloc[0]  # first row (cluster total) is the total count
    return col.map(lambda entry: f"..." if total != 0 else "...")

In [ ]:
records_num_df["..."] = records_num_df.loc[:, :].sum(axis=1).astype(int)

In [ ]:
records_num_df_final = records_num_df.apply(add_percentage, axis = 0)

In [ ]:
records_num_df_final

In [ ]:
records_num_df_final.to_csv("...", index = True)